# Sales Assistant

## Objective

This notebook demonstrates the following:

This notebook showcases a conversation with an Assistant equipped with multiple sales data files. Additionally, it illustrates the management of multiple user threads.

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant)
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)

## Time

You should expect to spend 10-15 minutes running this sample. 

## About this example

The objective of the provided Python file is to create an Azure OpenAI Assistant named "Sales Assistant" using the Azure OpenAI API. The assistant is designed to act act as a bot that can answer questions related to sales information including queries such as:

- Seller, customer, sales, and product information.
- Generate bar and chart of product sold
- Etc.

### Data
This sample uses the files in sub-folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.

## Before you begin

### Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
%pip install -r ../requirements.txt

### Parameters

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")  # make sure to have the .env file in the root directory of the project

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

should_cleanup: bool = False

## Run this Example

### Load the required libraries

In [ ]:
import html
import io
import time
import shelve
from datetime import datetime
from pathlib import Path
from typing import Iterable

import requests
import yfinance as yf
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta import Thread
from openai.types.beta.threads import Run
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

# List of assistants created
ai_assistants = []
# List of threads created
ai_threads = []
# List of files uploaded
ai_files = []

### Handle state

In [ ]:
def add_thread(thread):
    for item in ai_threads:
        if item.id == thread.id:
            return
    ai_threads.append(thread)
    print("Added thread: ", thread.id, len(ai_threads))

def check_if_thread_exists(user_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(user_id, None)

def store_thread(user_id, thread):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        add_thread(thread)
        threads_shelf[user_id] = thread.id

def clear_shelves():
    with shelve.open("assistant_db") as assistant_shelf:
        assistant_shelf.clear()
    with shelve.open("threads_db") as threads_shelf:
        threads_shelf.clear()

clear_shelves()

### Create an Azure OpenAI client

In [ ]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Get the latest stock price by ticker symbol using Yahoo Finance

In [ ]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    return stock.history(period="1d")["Close"].iloc[-1]

### Send an email using Logic Apps

In [ ]:
def send_logic_apps_email(to: str, content: str) -> None:
    try:
        json_payload = {"to": to, "content": html.unescape(content)}
        headers = {"Content-Type": "application/json"}
        response = requests.post(email_URI, json=json_payload, headers=headers)
        if response.status_code == 202:
            print("Email sent to: " + json_payload["to"])
    except:
        print("Failed to send email via Logic Apps")

### Define the Assistant tools

In [ ]:
tools_list = [
    {"type": "code_interpreter"}
]

### Upload the file(s)

In [ ]:
DATA_FOLDER = "data/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    print(path)
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_file = upload_file(client, filePath)
    ai_files.append(assistant_file)
    assistant_files.append(assistant_file)

file_ids = [file.id for file in assistant_files]
file_ids

### Create an Assistant and a Thread

In [ ]:
assistant = client.beta.assistants.create(
    name="Sales Assistant",
    instructions="You are a sales assistant. You can answer questions related to customer orders.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)
ai_assistants.append(assistant)
##thread = client.beta.threads.create()

### Handle and print Assistant Thread Messages

In [ ]:
def read_assistant_file(file_id:str):
    response_content = client.files.content(file_id)
    return response_content.read()

def print_messages(name: str, messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                if message.role == "user":
                    print(f"user: {name}:\n{item.text.value}\n")
                else:
                    print(f"{message.role}:\n{item.text.value}\n")
                file_annotations = item.text.annotations
                if file_annotations:
                    for annotation in file_annotations:
                        file_id = annotation.file_path.file_id
                        content = read_assistant_file(file_id)
                        print(f"Annotation Content:\n{str(content)}\n")
            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id                
                data_in_bytes = read_assistant_file(item.image_file.file_id)
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user Prompts

In [ ]:
def process_prompt(name, user_id, prompt: str) -> None:

    thread_id = check_if_thread_exists(user_id)

     # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with user_id {user_id}")
        thread = client.beta.threads.create()
        store_thread(user_id, thread)
        thread_id = thread.id
    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with user_id {user_id}")
        thread = client.beta.threads.retrieve(thread_id)
        add_thread(thread)

    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. Do not provide explanations for the answers."
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )

    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(name, messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(name, messages)
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":            
            pass
        else:
            time.sleep(5)

### Have a conversation with the Assistant

In [ ]:
process_prompt("John", "user_123", "What customers are in Florida?")

In [ ]:
process_prompt("Mary", "user_234", "What seller has had the most sales?")

In [ ]:
process_prompt("John", "user_123", "What is the most sold product?")

In [ ]:
process_prompt("Mary", "user_234", "Chart product sales by State.")

### Cleaning up

In [ ]:
def cleanup(client):
    print("Deleting: ", len(ai_assistants), " assistants.")
    for assistant in ai_assistants:
        print(client.beta.assistants.delete(assistant.id))
    print("Deleting: ", len(ai_threads), " threads.")
    for thread in ai_threads:
        print(client.beta.threads.delete(thread.id))
    print("Deleting: ", len(ai_files), " files.")
    for file in ai_files:
        print(client.files.delete(file.id))
        
cleanup(client)